In [1]:
import numpy as np
import model
import pandas as pd
import simpy

Example model, manufacturing system:

In [47]:
def bottle_filling(fbd,fsbd,lbd,lsbd,mo,frt,fsrt,lrt,lsrt,mt,p,mh):

    import numpy as np
    import simpy

        # Function to generate random breakdown times
    def breakdown_time():
        return np.random.exponential(fbd)  # Breakdown occurence distribution
        #return np.random.weibull(5,fbd)  # Breakdown occurence distribution

    def small_breakdown_time():
        return np.random.exponential(fsbd)  # Breakdown occurence distribution
        #return np.random.weibull(5,fsbd)  # Breakdown occurence distribution

    # Function to generate random breakdown times
    def labeller_breakdown_time():
        return np.random.exponential(lbd)  # Breakdown occurence distribution
        #return np.random.weibull(5,lbd)  # Breakdown occurence distribution

    def labeller_small_breakdown_time():
        return np.random.exponential(lsbd)  # Breakdown occurence distribution
        #return np.random.weibull(5,lsbd)  # Breakdown occurence distribution

    def maintenance_occurence_time():
        return mo  #Maintenance occurence distribution

    # Function to generate random repair times
    def repair_time():
        return np.random.exponential(frt)  #filler Repair time distribution
    def small_repair_time():
        return np.random.exponential(fsrt)  #filler small Repair time distribution

    def labeller_repair_time():
        return np.random.exponential(lrt)  #labeller Repair time distribution
    def labeller_small_repair_time():
        return np.random.exponential(lsrt)   #labeller small Repair time distribution
    def maintenance_time():
        return mt  # maintenance time

    # Bottling process
    def bottling(env, machine, stats):
        while True:
            yield env.timeout(1)  # Bottling takes 1 time unit

            if not machine.working:
                # If machine is not working due to breakdown, skip this cycle
                continue
            stats['total_bottles'] += 1

    # Breakdown process
    def breakdown(env, machine, stats):
        while True:
            yield env.timeout(breakdown_time())
            if machine.working:
                machine.working = False
                stats['filler_big_breakdowns'] += 1
                machine.down.succeed()  # Trigger the repair process
    def small_breakdown(env, machine, stats):
        while True:
            yield env.timeout(small_breakdown_time())
            if machine.working:
                machine.working = False
                stats['filler_small_breakdowns'] += 1
                machine.down2.succeed()  # Trigger the repair process
    # Breakdown process
    def labeller_breakdown(env, machine, stats):
        while True:
            yield env.timeout(labeller_breakdown_time())
            if machine.working:
                machine.working = False
                stats['labeller_big_breakdowns'] += 1
                machine.down3.succeed()  # Trigger the repair process
    def labeller_small_breakdown(env, machine, stats):
        while True:
            yield env.timeout(labeller_small_breakdown_time())
            if machine.working:
                machine.working = False
                stats['labeller_small_breakdowns'] += 1
                machine.down4.succeed()  # Trigger the repair process
    def maintenance(env, machine, stats):
        while True:
            yield env.timeout(maintenance_occurence_time())
            if machine.working:
                machine.working = False
                stats['Total maintenance occurences'] += 1
                machine.down5.succeed()  # Trigger the repair process

        # Repair process
    def repair(env, machine, stats):
        while True:
            yield machine.down
            repair_duration = repair_time()  # Get repair duration
            yield env.timeout(repair_duration)  # Wait for the repair duration
            stats['total_downtime'] += repair_duration
            stats['big_filler_downtime'] += repair_duration
            machine.working = True
            machine.down = env.event()  # Reset repair event

    def small_repair(env, machine, stats):
        while True:
            yield machine.down2
            repair_duration = small_repair_time()  # Get repair duration
            yield env.timeout(repair_duration)  # Wait for the repair duration
            stats['total_downtime'] += repair_duration
            stats['small_filler_downtime'] += repair_duration
            machine.working = True
            machine.down2 = env.event()  # Reset repair event

    # Repair process
    def labeller_repair(env, machine, stats):
        while True:
            yield machine.down3
            repair_duration = labeller_repair_time()  # Get repair duration
            yield env.timeout(repair_duration)  # Wait for the repair duration
            stats['total_downtime'] += repair_duration
            stats['labeller_big_downtime'] += repair_duration
            machine.working = True
            machine.down3 = env.event()  # Reset repair event

    def labeller_small_repair(env, machine, stats):
        while True:
            yield machine.down4
            repair_duration = labeller_small_repair_time()  # Get repair duration
            yield env.timeout(repair_duration)  # Wait for the repair duration
            stats['total_downtime'] += repair_duration
            stats['labeller_small_downtime'] += repair_duration
            machine.working = True
            machine.down4 = env.event()  # Reset repair event

    def maintenance_action(env, machine, stats):
        while True:
            yield machine.down5
            repair_duration = maintenance_time()  # Get repair duration
            yield env.timeout(repair_duration)  # Wait for the repair duration
            stats['total_downtime'] += repair_duration
            stats['Total_maintenance_time'] += repair_duration
            machine.working = True
            machine.down5 = env.event()  # Reset repair event

    # Simulation setup
    env = simpy.Environment()
    machine = simpy.Resource(env, capacity=1)  # Bottling machine resource
    machine.working = True  # Initially machine is working
    machine.down = env.event()  # Create event for filler breakdown
    machine.down2 = env.event()  # Create event for small filler breakdown
    machine.down3 = env.event()  # Create event for labeller breakdown
    machine.down4 = env.event()  # Create event for small labeller breakdown
    machine.down5 = env.event()  # Create event for machine maintenance

    stats = {'total_bottles': 0, 'filler_big_breakdowns': 0, 'total_downtime': 0, 'filler_small_breakdowns': 0,
             'labeller_small_breakdowns': 0, 'labeller_big_breakdowns': 0 ,'small_filler_downtime': 0, 'big_filler_downtime': 0
            , 'Total_maintenance_time': 0,'labeller_small_downtime':0,'labeller_big_downtime':0,'Total maintenance occurences':0}

    # Start processes
    env.process(bottling(env, machine, stats))

    #Filler inputs:
    env.process(repair(env, machine, stats))
    env.process(breakdown(env, machine, stats))
    env.process(small_repair(env, machine, stats))
    env.process(small_breakdown(env, machine, stats))
    #Labeller inputs:
    env.process(labeller_repair(env, machine, stats))
    env.process(labeller_breakdown(env, machine, stats))
    env.process(labeller_small_repair(env, machine, stats))
    env.process(labeller_small_breakdown(env, machine, stats))
    #Maintenance inputs:
    env.process(maintenance_action(env, machine, stats))
    env.process(maintenance(env, machine, stats))

    # Run simulation
    simulation_time = 168*60*6 #Simulation time in minutes (6 weeks)
    env.run(until=simulation_time)  # Run the simulation for amount of simulation time
    
    ans = stats['total_bottles']*100-p*30*168*6-10000*6*mh

    return ans/1000

#How to use the function: bottle_filling(fbd,fsmb,lbd,lsbd,mo,frt,fsrt,lrt,lsrt,mt);
#fbd:filler breakdown rate ,fsmb: filler small breakdown rate,lbd:labeller breakdown rate,
#lsbd: labeller small breakdown rate ,mo: maintenance rate ,frt: filler big breakdown repair time,
#fsrt: filler small breakdown repair time,lrt: labeller repair time,lsrt: labeller small breakdown repair time,
#mt: maintenance time. All inputs are epxonential with the expected value being the input, except for maintenance time and
#maintenance frequency, these are a specific time between maintenance and a specific time for maintenance, since they usually
#can be determined by operations. Everything is in minutes, the output is the amount of minutes of filling line operation in 4 weeks

In [3]:
def create_single_X(k,n0):
    ll = []
    for i in range(n0):
        ll.append(bottle_filling(k[0],k[1],k[2],k[3],k[4],k[5],k[6],k[7],k[8],k[9],k[10],k[11]))
    return ll
                   
def create_X(k, n0):
    l = []
    for i in range(len(k)):        
        l.append(create_single_X(k[i],n0))
    return np.array(l)
        
def sample_variance(X):
    n = len(X[0])
    k = len(X)    
    som = 0
    for i in range(n):
        for j in range(k):
            t = np.subtract(X[j,i]-np.mean(X[:,i]),np.mean(X[j,:]))+np.mean(X)
            som += 2*t**2
    return som/(k-1)/(n-1)   


In [49]:
k1 = [24*60,60,36*60,45,24*7*60,2*60,2,4*60,4,4*60,2,4]
k2 = [24*60,60,36*60,45,24*7*60,2*60,1,4*60,2,4*60,3,4]
k3 = [48*60,60,72*60,45,24*7*60,2*60,2,4*60,4,4*60,2,8]
k4 = [72*60,60,72*60,45,24*7*60,2*60,1,4*60,2,4*60,3,8]
#k5 = [72*60,60,72*60,45,24*7*60,2*60,1,4*60,2,4*60,3,12]
kall = [k1,k2,k3,k4]

In [50]:
n_0=20

X1 = pd.DataFrame(create_X(kall,n_0))
res = pd.DataFrame({'mean': X1.mean(axis=1), 'var': X1.var(axis=1)})



In [101]:
d_star = 20
P_star = 0.95
ksystem=4
m=2
h2 = 2.071

In [102]:
res['N_i'] = res.apply(lambda row: max(np.ceil((h2**2)*row["var"]/d_star**2), n_0 + 1), axis=1).astype(int)

#res.shape[0]

In [53]:
res

,iter,mean,var,N_i
0,1,4221.465,32702.479447,129
1,2,4471.880,19075.223158,76
2,3,4364.705,10714.846605,43
3,4,4626.905,10809.879868,43


In [129]:
res_stage2 = []

for i in range(res.shape[0]):
    h_list = create_single_X(kall[i],(res['N_i'][i]-n_0))
    res_stage2.append(np.mean(h_list))
    
# print(res_stage2)

In [130]:
res['mean(N_i - n_0)'] = res_stage2
display(res)

,iter,mean,var,N_i,mean(N_i - n_0)
0,1,4221.465,32702.479447,351,4234.829366
1,2,4471.880,19075.223158,205,4428.197838
2,3,4364.705,10714.846605,115,4344.129474
3,4,4626.905,10809.879868,116,4618.499792


In [135]:
resm = res.drop(columns = ['iter']).copy()
resm.insert(0, 'pol', [2,2,3,3])
resm.insert(1, 'mh', [4,8,3,8])
display(resm)

,pol,mh,mean,var,N_i,mean(N_i - n_0)
0,2,4,4221.465,32702.479447,351,4234.829366
1,2,8,4471.880,19075.223158,205,4428.197838
2,3,3,4364.705,10714.846605,115,4344.129474
3,3,8,4626.905,10809.879868,116,4618.499792


In [136]:
resm['square_brac'] = 1-(resm['N_i']-n_0)*d_star**2/(h2**2*resm['var'])
resm['square_root_arg'] = 1 - resm['N_i']/n_0*resm['square_brac'] 
resm['square_root'] = np.sqrt(resm['square_root_arg'])
resm['W_i1'] = round((n_0/resm['N_i'])*(1+resm['square_root']),3)
resm['W_i2'] = round(1-resm['W_i1'], 3)
resm['X_tilda'] = round((resm['W_i1']*resm['mean'] + resm['W_i2']*resm['mean(N_i - n_0)'])*1000,2)
resm = resm.drop(['square_brac', 'square_root_arg','square_root'], axis=1)
display(resm)

,pol,mh,mean,var,N_i,mean(N_i - n_0),W_i1,W_i2,X_tilda
0,2,4,4221.465,32702.479447,351,4234.829366,0.064,0.936,4233974.05
1,2,8,4471.880,19075.223158,205,4428.197838,0.112,0.888,4433090.24
2,3,3,4364.705,10714.846605,115,4344.129474,0.186,0.814,4347956.52
3,3,8,4626.905,10809.879868,116,4618.499792,0.183,0.817,4620037.94


In [137]:
resm['mean(N_i - n_0)'] = round((resm['mean(N_i - n_0)'])*1000,2)
resm['mean'] = round(resm['mean']*1000,2)
resm['var'] = round(resm['var'],2)
resm['X_tilda'] = round(resm['X_tilda'])

In [138]:
resm['var'] = round(resm['var'],2)
resm['X_tilda'] = round(resm['X_tilda'],)

In [139]:
resm

,pol,mh,mean,var,N_i,mean(N_i - n_0),W_i1,W_i2,X_tilda
0,2,4,4221465.0,32702.48,351,4234829.37,0.064,0.936,4233974.0
1,2,8,4471880.0,19075.22,205,4428197.84,0.112,0.888,4433090.0
2,3,3,4364705.0,10714.85,115,4344129.47,0.186,0.814,4347957.0
3,3,8,4626905.0,10809.88,116,4618499.79,0.183,0.817,4620038.0


In [140]:
print(resm.to_latex(index=True,
                  formatters={"name": str.upper}))

\begin{tabular}{lrrrrrrrrr}
\toprule
 & pol & mh & mean & var & N_i & mean(N_i - n_0) & W_i1 & W_i2 & X_tilda \\
\midrule
0 & 2 & 4 & 4221465.000000 & 32702.480000 & 351 & 4234829.370000 & 0.064000 & 0.936000 & 4233974.000000 \\
1 & 2 & 8 & 4471880.000000 & 19075.220000 & 205 & 4428197.840000 & 0.112000 & 0.888000 & 4433090.000000 \\
2 & 3 & 3 & 4364705.000000 & 10714.850000 & 115 & 4344129.470000 & 0.186000 & 0.814000 & 4347957.000000 \\
3 & 3 & 8 & 4626905.000000 & 10809.880000 & 116 & 4618499.790000 & 0.183000 & 0.817000 & 4620038.000000 \\
\bottomrule
\end{tabular}



### Second Round - finding optimal

In [160]:
res2 = resm[['pol', 'mh', 'mean', 'var']].copy()

In [161]:
res2 = res2.iloc[[1,3]]

In [162]:
h1 = 2.453
res2['N_i'] = res.apply(lambda row: max(np.ceil((h1**2)*row["var"]/d_star**2), n_0 + 1), axis=1).astype(int)


In [164]:
res_stage22 = []

for i in [1,3]:
    h_list2 = create_single_X(kall[i],(res2['N_i'][i]-n_0))
    res_stage22.append(np.mean(h_list2))
    
# print(res_stage2)

In [165]:
sum(res2['N_i'])

450

In [166]:
res2['mean(N_i - n_0)'] = res_stage22
res2['mean']=res2['mean']/1000

res2['square_brac'] = 1-(res2['N_i']-n_0)*d_star**2/(h2**2*res2['var'])
res2['square_root_arg'] = 1 - res2['N_i']/n_0*res2['square_brac'] 
res2['square_root'] = np.sqrt(res2['square_root_arg'])
res2['W_i1'] = round((n_0/res2['N_i'])*(1+res2['square_root']),3)
res2['W_i2'] = round(1-res2['W_i1'], 3)
res2['X_tilda'] = (res2['W_i1']*res2['mean'] + res2['W_i2']*res2['mean(N_i - n_0)'])
res2 = res2.drop(['square_brac', 'square_root_arg','square_root'], axis=1)
display(res2)

,pol,mh,mean,var,N_i,mean(N_i - n_0),W_i1,W_i2,X_tilda
1,2,8,4471.880,19075.22,287,4421.459026,0.231,0.769,4433.106271
3,3,8,4626.905,10809.88,163,4643.243636,0.332,0.668,4637.819209


In [167]:
res2['mean(N_i - n_0)'] = round((res2['mean(N_i - n_0)'])*1000,2)
res2['mean'] = round(res2['mean']*1000,2)
res2['var'] = round(res2['var'],2)
res2['X_tilda'] = round(res2['X_tilda']*1000.2)

In [168]:
display(res2)

,pol,mh,mean,var,N_i,mean(N_i - n_0),W_i1,W_i2,X_tilda
1,2,8,4471880.0,19075.22,287,4421459.03,0.231,0.769,4433993.0
3,3,8,4626905.0,10809.88,163,4643243.64,0.332,0.668,4638747.0


In [169]:
print(res2.to_latex(index=True,
                  formatters={"name": str.upper}))

\begin{tabular}{lrrrrrrrrr}
\toprule
 & pol & mh & mean & var & N_i & mean(N_i - n_0) & W_i1 & W_i2 & X_tilda \\
\midrule
1 & 2 & 8 & 4471880.000000 & 19075.220000 & 287 & 4421459.030000 & 0.231000 & 0.769000 & 4433993.000000 \\
3 & 3 & 8 & 4626905.000000 & 10809.880000 & 163 & 4643243.640000 & 0.332000 & 0.668000 & 4638747.000000 \\
\bottomrule
\end{tabular}

